In [ ]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName):
    path = "old_results/batch/dynamic/gridsearch_amanda_dynamic_EVL-{}.txt".format(clfName)
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    #datasets=[setup.loadKeystroke]
    #batches=8
    #datasets=[setup.loadNOAADataset]
    #batches=50
    
    arrClfName = ['LP', 'KNN']#['SGD', 'NB', 'RF', 'LP', 'KNN']
    for clfName in arrClfName:
        print("**************** BEGIN of {} results ****************".format(clfName))
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            batches=100
            if i==len(datasets)-2:
                batches=50
            elif i==len(datasets)-1:
                batches=8
                
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)
            
            initialLabeledData = int(0.05*len(dataLabels))
            sizeOfBatch = int((len(dataLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize],
                             "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData], "clfName":[clfName]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(dataValues, dataLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName)
        print("******** END of {} results ********".format(clfName))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP results ****************
One Class Diagonal Translation. 2 Dimensional data.: 100 batches of 152 instances
99.6967 {'K': 2, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.7362 {'K': 3, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.7494 {'K': 4, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.7428 {'K': 5, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.7428 {'K': 6, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.7231 {'K': 7, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 152}
99.723 {'K': 8, 'batches': 100, 'clfName': 'LP',